In [1]:
import os, sys
import pandas as pd
import numpy as np
import xarray as xr
from itertools import product
from datetime import datetime, timedelta
import argparse
import logging
from scipy.special import gamma, factorial #gamma function
sys.path.append('/g/data/er4/zk6340/code/lmoments3')
import lmoments3 as lm #calculate l-moments
%matplotlib inline

In [2]:
## dir folder path
dir_in = '/g/data/er4/zk6340/Hydro_projection/data_flood_scenario_pr'
dir_out = '/g/data/er4/zk6340/code/Script_Hydro-projections'

# parameter
parameter = 'pr'

# Which cluster
clusters = {'CS':1,'EC':2,'MB':4,'MN':5,'R':6,'SS':7,'SSWF':8,'WT':9}
which_cluster = clusters['MB']

# return period
yT = 20

# historical period
hist_st = '19760101'
hist_end = '20051231'

# projection period
yr_st = '20160101'
yr_end = '20451231'

# Participating GCMs, Bias correction approaches, Emission scenarios
gcms = ['CNRM-CERFACS-CNRM-CM5','CSIRO-BOM-ACCESS1-0','MIROC-MIROC5','NOAA-GFDL-GFDL-ESM2M']
bias_corr = ['CSIRO-CCAM-r3355-r240x120-ISIMIP2b-AWAP','r240x120-ISIMIP2b-AWAP', 'r240x120-MRNBC-AWAP', 'r240x120-QME-AWAP']
emission = ['rcp45','rcp85']

In [3]:
#Function to get to the historical/scenario file
def get_file(parameter,which_gcm,which_emission,which_bias_corr, which_metric, yr_end, yr_st):
    base_filename = '%s_AUS-5_%s_%s_r1i1p1_%s_%s_%s-%s.nc'%(parameter,which_gcm,which_emission,which_bias_corr,which_metric,yr_end,yr_st)
    return(base_filename)

In [4]:
#Function to get percentage change for 'Mean' and 'Max'
def percentage_change(parameter, gcm, scenario, bias_correction, statistics, hist_end, hist_st, yr_end, yr_st):
    # Historical period
    filename_hist = get_file(parameter, gcm,'historical', bias_correction, statistics, hist_end, hist_st)
    ds_hist = xr.open_dataset(os.path.join(dir_in,filename_hist)) 
    ds_hist_cluster = ds_hist[parameter].where(mask == which_cluster)
    ds_hist_cluster_spatial_mean = ds_hist_cluster.NRM_cluster.mean().item(0)
    # Future scenario
    filename_scenario = get_file(parameter, gcm, scenario, bias_correction, statistics,yr_end,yr_st)
    ds_scenario = xr.open_dataset(os.path.join(dir_in,filename_scenario)) 
    ds_scenario_cluster = ds_scenario[parameter].where(mask == which_cluster)
    ds_scenario_cluster_spatial_mean = ds_scenario_cluster.NRM_cluster.mean().item(0)   
    percent_change = ((ds_scenario_cluster_spatial_mean-ds_hist_cluster_spatial_mean)*100)/ds_hist_cluster_spatial_mean  
    return(percent_change)

In [5]:
#Function to calculate return period: GEV distribution
def gev_return_period(data, T):
    lamda1= lm.lmom_ratios(data, nmom=4)[0]
    lamda2= lm.lmom_ratios(data, nmom=4)[1]
    lamda3= lm.lmom_ratios(data, nmom=4)[2]
    lamda4= lm.lmom_ratios(data, nmom=4)[3]
    c=(2*lamda2/(lamda3+3*lamda2))-(np.log(2)/np.log(3))
    k=7.859*c+2.9554*c**2
    alpha=k*lamda2/(gamma(1+k)*(1-2**(-k)))
    zeta=lamda1-alpha/k*(1-(gamma(1+k)))
    qt=zeta+(alpha/k)*(1-(-np.log(1-1/T))**k)
    return(qt)

In [6]:
#Function to get annual maximum timeseries for a grid cell
def ts_data(dataset, lat, lon, parameter):
    ds = dataset.sel(lat = lat, lon = lon, method = 'nearest')[parameter].values
    return(ds)

In [7]:
# Read cluster mask
mask = xr.open_dataset(os.path.join(dir_in,'NRM_clusters.nc'))

In [8]:
# picking up grid cells for a particular cluster
grid_cells = mask.where(mask == which_cluster).NRM_cluster.to_dataframe().reset_index()
grid_cells = grid_cells.dropna(axis=0).reset_index()

In [9]:
values = [(i, k, x) for i, k, x in product(gcms,emission, bias_corr)]

In [ ]:
values

### Faster code

In [10]:
change_daily_mean = percentage_change(parameter, values[27][0], values[27][1], values[27][2], 'Mean', hist_end, hist_st, yr_end, yr_st)

#Change in max
change_daily_max = percentage_change(parameter, values[27][0], values[27][1], values[27][2], 'Max', hist_end, hist_st, yr_end, yr_st)  

#Change in return period
# Annual Max - for historical period
filename_hist_annual_max = get_file(parameter,values[27][0],'historical',values[27][2],'Annual_Max',hist_end,hist_st)
ds_hist_annual_max = xr.open_dataset(os.path.join(dir_in,filename_hist_annual_max))
# Annual Max - for scenario period
filename_scenario_annual_max = get_file(parameter,values[27][0],values[27][1],values[27][2],'Annual_Max',yr_end,yr_st)
ds_scenario_annual_max = xr.open_dataset(os.path.join(dir_in,filename_scenario_annual_max))  
# calculate retrun period for the historical period
all_yT = []
for i in range (0, len(grid_cells)): 
    hist_yT_point = gev_return_period(ts_data(ds_hist_annual_max, grid_cells.lat[i], grid_cells.lon[i], parameter), yT)
    scenario_yT_point = gev_return_period(ts_data(ds_scenario_annual_max, grid_cells.lat[i], grid_cells.lon[i], parameter), yT)       
    yT_point = {'hist_yT':hist_yT_point,'future_yT':scenario_yT_point}
    all_yT.append(yT_point)
all_return_period = pd.DataFrame(all_yT)   
change_return_period = ((all_return_period['future_yT'].mean()-all_return_period['hist_yT'].mean())*100)/all_return_period['hist_yT'].mean()   
dd = {'GCM':values[27][0],'Bias Correction':values[27][2],'Emission':values[27][1],'Change in Daily Mean':change_daily_mean,'Change in daily Max':change_daily_max, 'Change in Return Period':change_return_period}

In [11]:
dd

{'GCM': 'NOAA-GFDL-GFDL-ESM2M',
 'Bias Correction': 'r240x120-QME-AWAP',
 'Emission': 'rcp45',
 'Change in Daily Mean': -17.068917532352838,
 'Change in daily Max': -3.203780330827212,
 'Change in Return Period': -2.309028651144917}

In [12]:
all_return_period

,hist_yT,future_yT
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
18122,66.939304,56.862671
18123,69.320159,58.651668
18124,66.322657,56.476517
18125,66.107530,56.299584


In [13]:
hist_yT_point

66.12902016388372

In [23]:
all_return_period['future_yT'].min()

36.47752452972815

In [24]:
all_return_period['hist_yT'].min()

37.269839489960866

In [ ]:
grid_cells

In [18]:
data_test = ds_hist_annual_max.sel(lat = -32.25, lon = 145.65, method = 'nearest')[parameter].values

In [19]:
data_test

array([ 46.3819854 ,  26.94592103,  28.9420981 ,  46.3819854 ,
        27.72732647,  26.94592103,  18.84902897,  36.901809  ,
        43.18356682,  77.58088075,  32.91497687,  62.61413936,
        29.78134621,  35.35311492,  25.81488965,  14.99467702,
        44.43557672,  35.86198362,  26.56351766, 110.88779122,
        46.3819854 ,  19.95832315,  53.5068512 ,  40.78430654,
        31.08606404,  24.0342943 ,  28.53136556,  40.20562079,
        37.97175642,  41.37131805])

In [25]:
gev_return_period(data_test, 20)

66.93930365936576

In [ ]:
caluster_flood_indicator=[]
for val in enumerate(values):   
#for val in enumerate(values[0]):  
    #Change in mean
    change_daily_mean = percentage_change(parameter, val[1][0], val[1][1], val[1][2], 'Mean', hist_end, hist_st, yr_end, yr_st)

    #Change in max
    change_daily_max = percentage_change(parameter, val[1][0], val[1][1], val[1][2], 'Max', hist_end, hist_st, yr_end, yr_st)  
    
    #Change in return period
    # Annual Max - for historical period
    filename_hist_annual_max = get_file(parameter,val[1][0],'historical',val[1][2],'Annual_Max',hist_end,hist_st)
    ds_hist_annual_max = xr.open_dataset(os.path.join(dir_in,filename_hist_annual_max))
    # Annual Max - for scenario period
    filename_scenario_annual_max = get_file(parameter,val[1][0],val[1][1],val[1][2],'Annual_Max',yr_end,yr_st)
    ds_scenario_annual_max = xr.open_dataset(os.path.join(dir_in,filename_scenario_annual_max))  
    # calculate retrun period for the historical period
    all_yT = []
    for i in range (0, len(grid_cells)): 
        hist_yT_point = gev_return_period(ts_data(ds_hist_annual_max, grid_cells.lat[i], grid_cells.lon[i], parameter), yT)
        scenario_yT_point = gev_return_period(ts_data(ds_scenario_annual_max, grid_cells.lat[i], grid_cells.lon[i], parameter), yT)       
        yT_point = {'hist_yT':hist_yT_point,'future_yT':scenario_yT_point}
        all_yT.append(yT_point)
    all_return_period = pd.DataFrame(all_yT)   
    change_return_period = ((all_return_period['future_yT'].mean()-all_return_period['hist_yT'].mean())*100)/all_return_period['hist_yT'].mean()   
    dd = {'GCM':val[1][0],'Bias Correction':val[1][2],'Emission':val[1][1],'Change in Daily Mean':change_daily_mean,'Change in daily Max':change_daily_max, 'Change in Return Period':change_return_period}
    cluster_flood_indicator.append(dd)
    break # just want to check one loop/one ensemble

In [ ]:
cluster_flood_indicator

In [ ]:
values[10][0]

#### Slow code

In [ ]:
cluster_flood_indicator=[]
for val in enumerate(values):   
#for val in enumerate(values[0]):  
    #Change in mean
    change_daily_mean = percentage_change(parameter, val[1][0], val[1][1], val[1][2], 'Mean', hist_end, hist_st, yr_end, yr_st)

    #Change in max
    change_daily_max = percentage_change(parameter, val[1][0], val[1][1], val[1][2], 'Max', hist_end, hist_st, yr_end, yr_st)  
    
    #Change in return period
    # Annual Max - for historical period
    filename_hist_annual_max = get_file(parameter,val[1][0],'historical',val[1][2],'Annual_Max',hist_end,hist_st)
    ds_hist_annual_max = xr.open_dataset(os.path.join(dir_in,filename_hist_annual_max))
    # calculate retrun period for the historical period
    hist_yT = []
    for i in range (0, len(grid_cells)): 
        hist_yT_point = gev_return_period(ts_data(ds_hist_annual_max, grid_cells.lat[i], grid_cells.lon[i], parameter), yT)
        hist_yT.append(hist_yT_point)
    hist_yT_mean = pd.DataFrame(hist_yT).mean()[0]
    # Annual Max - for scenario period
    filename_scenario_annual_max = get_file(parameter,val[1][0],val[1][1],val[1][2],'Annual_Max',yr_end,yr_st)
    ds_scenario_annual_max = xr.open_dataset(os.path.join(dir_in,filename_scenario_annual_max))
    # calculate retrun period for the scenario period
    scenario_yT = []
    for j in range (0, len(grid_cells)): 
        scenario_yT_point = gev_return_period(ts_data(ds_scenario_annual_max, grid_cells.lat[j], grid_cells.lon[j], parameter), yT)
        scenario_yT.append(scenario_yT_point)
    scenario_yT_mean = pd.DataFrame(scenario_yT).mean()[0]
    change_return_period = ((scenario_yT_mean-hist_yT_mean)*100)/hist_yT_mean
    
    dd = {'GCM':val[1][0],'Bias Correction':val[1][2],'Emission':val[1][1],'Change in Daily Mean':change_daily_mean,'Change in daily Max':change_daily_max, 'Change in Return Period':change_return_period}
    cluster_flood_indicator.append(dd)
    break # just want to check one loop/one ensemble

#### Mean of change

In [ ]:
cluster_flood_indicator=[]
for val in enumerate(values):   
#for val in enumerate(values[0]):  
    #Change in mean
    change_daily_mean = percentage_change(parameter, val[1][0], val[1][1], val[1][2], 'Mean', hist_end, hist_st, yr_end, yr_st)

    #Change in max
    change_daily_max = percentage_change(parameter, val[1][0], val[1][1], val[1][2], 'Max', hist_end, hist_st, yr_end, yr_st)  
    
    #Change in return period
    # Annual Max - for historical period
    filename_hist_annual_max = get_file(parameter,val[1][0],'historical',val[1][2],'Annual_Max',hist_end,hist_st)
    ds_hist_annual_max = xr.open_dataset(os.path.join(dir_in,filename_hist_annual_max))
    # Annual Max - for scenario period
    filename_scenario_annual_max = get_file(parameter,val[1][0],val[1][1],val[1][2],'Annual_Max',yr_end,yr_st)
    ds_scenario_annual_max = xr.open_dataset(os.path.join(dir_in,filename_scenario_annual_max))  
    # calculate retrun period for the historical period
    change_yT = []
    for i in range (0, len(grid_cells)): 
        hist_yT_point = gev_return_period(ts_data(ds_hist_annual_max, grid_cells.lat[i], grid_cells.lon[i], parameter), yT)
        scenario_yT_point = gev_return_period(ts_data(ds_scenario_annual_max, grid_cells.lat[i], grid_cells.lon[i], parameter), yT)
        diff_yT = ((scenario_yT_point-hist_yT_point)*100)/hist_yT_point
        change_yT.append(diff_yT)
    change_return_period = pd.DataFrame(change_yT).mean()[0]   
    dd = {'GCM':val[1][0],'Bias Correction':val[1][2],'Emission':val[1][1],'Change in Daily Mean':change_daily_mean,'Change in daily Max':change_daily_max, 'Change in Return Period':change_return_period}
    cluster_flood_indicator.append(dd)
    break # just want to check one loop/one ensemble

In [ ]:
cluster_flood_indicator